In [1]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
load_dotenv()

foundary_project_endpoint=os.getenv("FOUNDARY_PROJECT_ENDPOINT")
model_deployment_name=os.getenv("MODEL_DEPLOYMENT_NAME")

In [5]:
project_client = AIProjectClient(
    endpoint=foundary_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Setting up a simple validator Agent

In [11]:
from azure.ai.projects.models import PromptAgentDefinition

agent_name = "prpo-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a PR-PO functional agent. You understand the process well and can answer questions around the entire PR PO process for a CPG company like Mars "
    ),
)

# preparing the agent id
print(f"Agent created (id: {agent,id}, name: {agent.name}, version: {agent.version})")


Agent created (id: ({'metadata': {}, 'object': 'agent.version', 'id': 'prpo-agent:1', 'name': 'prpo-agent', 'version': '1', 'description': '', 'created_at': 1767090362, 'definition': {'kind': 'prompt', 'model': 'gpt-4o', 'instructions': 'You are a PR-PO functional agent. You understand the process well and can answer questions around the entire PR PO process for a CPG company like Mars '}}, <built-in function id>), name: prpo-agent, version: 1)


### Creating a conversation object for agent chat system 

In [12]:
# creating an openai client first
openai_client = project_client.get_openai_client()

# createing a conversation to use with the agent
conversation = openai_client.conversations.create()

print(f"Created conversation with id {conversation.id}")

Created conversation with id conv_d8be4e14739207e300FekJsYxdHbiMJJ1MI7t5xsLmn80g0yVO


### Chat with the Agent

In [ ]:
chat = True

while chat:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        chat = False
        print("Exiting Chat, Goodbye")

    else:
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body= {
                "agent": {
                    "name": agent_name,
                    "type": "agent_reference"
                }
            },
            input = user_input
        )

        print(f"PR_PO_Functional_Consultant: {response.output_text}")